Adapted from https://github.com/philferriere/dlwin

additional references:
 - https://stackoverflow.com/questions/42728877/using-pymc3-on-windows-10-theano-cannot-import-name-floatx?noredirect=1#comment72639271_42728877
 - https://stackoverflow.com/a/44503723/2123721
 - http://deeplearning.net/software/libgpuarray/installation.html#windows-specific-instructions

    Define sysenv variable PYTHON_HOME with the value C:\<ANACONDA_DIRECTORY>\
    Add %PYTHON_HOME%, %PYTHON_HOME%\Scripts, and %PYTHON_HOME%\Library\bin to PATH

conda create --yes -n my_custom_py36_env python=3.6 anaconda numpy scipy mkl-service m2w64-toolchain libpython jupyter

activate my_custom_py36_env

conda install -c rdonnelly Theano

5. Add the mingw64 that exists in the anaconda environment to system environment PATH

C:\Users\ein\Anaconda3\envs\my_custom_py36_env\Library\mingw-w64\bin

conda install pygpu==0.6.2 nose theano

conda install -c conda-forge theano pygpu

In [1]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

Can not use cuDNN on context None: Disabled by dnn.enabled flag
Preallocating 1638/2048 Mb (0.800000) on cuda0
Mapped name None to device cuda0: GeForce GTX 960M (0000:01:00.0)


[GpuElemwise{exp,no_inplace}(<GpuArrayType<None>(float64, (False,))>), HostFromGpu(gpuarray)(GpuElemwise{exp,no_inplace}.0)]


Looping 1000 times took 1.037000 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the gpu
